In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import tensorflow as tf
import tensorflow_probability as tfp
import h5py
import random
from sklearn.model_selection import train_test_split
import pandas as pd
from DataMaker import HDF5ImageGenerator

In [2]:
import photoz_utils

In [3]:
tfpl = tfp.layers
tf1 = tf.compat.v1
tfd = tfp.distributions

In [4]:
from tensorflow_probability.python.distributions import kl_divergence

In [5]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2022-10-07 01:43:13.182792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 01:43:13.193098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 01:43:13.193721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-07 01:43:13.194904: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
LATENT_DIM = 16
IMAGE_SHAPE = (5, 127, 127)
BATCH_SIZE = 128
EPOCHS = 90
GB_LIMIT = 30
CHECKPOINTS_TO_SAVE = 1

In [7]:
args_gen = {'X_key': 'image',
    'y_key': 'specz_redshift',
    'scaler': True,
    'labels_encoding': False,
    'batch_size': BATCH_SIZE,
    'mode': 'train',
    'shuffle': True}

train_gen = HDF5ImageGenerator(src='/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training.hdf5', **args_gen)
val_gen = HDF5ImageGenerator(src='/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation.hdf5', **args_gen)
test_gen = HDF5ImageGenerator(src='/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing.hdf5', **args_gen)

In [8]:
n = BATCH_SIZE
l = len(train_gen._indices)

In [9]:
def posterior_mean_field(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))

    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype = dtype, initializer = lambda shape, dtype: random_gaussian_initializer(shape, dtype), trainable = True),
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc = t[ ..., : n],
                       scale = 0 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims = 1)),
    ])

def prior_trainable(kernel_size: int, bias_size: int, dtype: any) -> tf.keras.Model:
    """Specify the prior over `keras.layers.Dense` `kernel` and `bias`."""
    n = kernel_size + bias_size

    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype = dtype),  # Returns a trainable variable of shape n, regardless of input
        tfp.layers.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc = t, scale = 0.1),
            reinterpreted_batch_ndims = 1)),
    ])

def random_gaussian_initializer(shape, dtype):
    n = int(shape / 2)
    loc_norm = tf.random_normal_initializer(mean = 0., stddev = 0.001)
    loc = tf.Variable(
        initial_value = loc_norm(shape = (n, ), dtype = dtype)
    )
    scale_norm = tf.random_normal_initializer(mean = -3., stddev = 0.1)
    scale = tf.Variable(
        initial_value = scale_norm(shape = (n, ), dtype = dtype)
    )
    return tf.concat([loc, scale], 0)

kl = 0

kl_divergence_function = 0 # lambda q, p, _: kl_divergence(q, p) / n

In [10]:
import keras
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

In [17]:
model_variational = Sequential([
    InputLayer(input_shape = (5, 127, 127)),
    Conv2D(32, kernel_size = (3, 3), activation = 'tanh', padding = 'same', data_format = 'channels_first'),
    MaxPooling2D(pool_size = (2, 2), data_format = 'channels_first'),
    Conv2D(64, kernel_size = (2, 2), activation = 'tanh', padding = 'same', data_format = 'channels_first'),
    MaxPooling2D(pool_size = (2, 2), data_format = 'channels_first'),
    Conv2D(128, kernel_size = (2, 2), activation = 'tanh', padding = 'same', data_format = 'channels_first'),
    MaxPooling2D(pool_size = (2, 2), data_format = 'channels_first'),
    Conv2D(256, kernel_size = (2, 2), activation = 'tanh', padding = 'same', data_format = 'channels_first'),
    MaxPooling2D(pool_size = (2, 2), data_format = 'channels_first'),
    Conv2D(256, kernel_size = (2, 2), activation = 'tanh', padding = 'same', data_format = 'channels_first'),
    MaxPooling2D(pool_size = (2, 2), data_format = 'channels_first'),
    Conv2D(512, kernel_size = (3, 3), activation = 'relu', padding = 'same', data_format = 'channels_first'),
    Conv2D(512, kernel_size = (2, 2), activation = 'relu', padding = 'same', data_format = 'channels_first'),
    Flatten(),
    Dropout(0.2),
    tfpl.DenseVariational(512, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = kl),
    tfpl.DenseVariational(128, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = kl),
    tfpl.DenseVariational(128, 
                                      make_posterior_fn = posterior_mean_field,
                                      make_prior_fn = prior_trainable,
                                      kl_weight = kl),
    Dense(tfpl.IndependentNormal.params_size(1)),
    tfpl.IndependentNormal(1)
])

model_variational.build((n, 5, 127, 127))

model_variational.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 32, 127, 127)      1472      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 32, 63, 63)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 64, 63, 63)        8256      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 64, 31, 31)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 128, 31, 31)       32896     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 128, 15, 15)     

In [18]:
def loss_function(targets, estimated_distribution):
    return - estimated_distribution.log_prob(targets) / (1 + targets)

model_variational.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = loss_function, metrics = [keras.metrics.RootMeanSquaredError()])

In [19]:
import tensorboard

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

model_name = 'HSC_v6_BCNN_v7_nobc'

checkpoint_filepath = os.path.join('/models/', model_name)

weights_path = os.path.join('/models/', model_name) + '_weights'

log_directory = os.path.join('/logs/', model_name)

logs_callback = TensorBoard(log_dir = log_directory)

weights_callback = ModelCheckpoint(filepath = os.path.join(checkpoint_filepath, 'weights_epoch{epoch}.hdf5'),
                                   save_freq = int(EPOCHS / CHECKPOINTS_TO_SAVE) * int(l / BATCH_SIZE),
                                   save_weights_only = True, save_best_only = True)

LR_callback = ReduceLROnPlateau()

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_directory, histogram_freq = 1)

In [20]:
class KLCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        if epoch == 5:
            kl = 1 / n
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

In [ ]:
model_variational.fit(train_gen, epochs = EPOCHS, shuffle = True, verbose = 1, validation_data = val_gen, callbacks = [tensorboard_callback, logs_callback, LR_callback, weights_callback])

Epoch 1/90
 429/1567 [=======>......................] - ETA: 14:22 - loss: nan - root_mean_squared_error: nan

In [ ]:
model_variational.save_weights(weights_path.format(epoch = 5))

In [ ]:
model_variational.load_weights(weights_path)

In [ ]:
t = 2000

hf_train = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_training.hdf5', 'r')
y_train = np.asarray(hf_train['specz_redshift'][0:])[..., None]
object_id_train = np.asarray(hf_train['object_id'][0:])
hf_train.close()

In [ ]:
hf_test = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing.hdf5', 'r')
x_test = np.asarray(np.transpose(hf_test['image'][0:],(0,1,2,3)))
x_test = x_test[:t]
x_test = np.true_divide(x_test, 4.16)
y_test = np.asarray(hf_test['specz_redshift'][0:])[..., None][:t]
object_id = np.asarray(hf_test['object_id'][0:])[:t]
hf_test.close()

hf_validation = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_validation.hdf5', 'r')
y_validation = np.asarray(hf_validation['specz_redshift'][0:])[..., None]
object_id_validation = np.asarray(hf_validation['object_id'][0:])
hf_validation.close()

In [ ]:
photoz = []

for i in range(t):
    photoz.append(model_variational(np.expand_dims(x_test[i], axis = 0)).mean()[0][0])
    print(i)

photoz = np.asarray(photoz).astype('float32')

In [ ]:
photoz_err = []

for i in range(t):
    photoz_err.append(model_variational(np.expand_dims(x_test[i], axis = 0)).stddev()[0][0])
    print(i)

photoz_err = np.asarray(photoz_err).astype('float32')

photoz = []

for i in range(t):
    photoz.append(model_variational(np.array([test_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).mean()[0][0])

photoz = np.asarray(photoz).astype('float32')

photoz_train = []

for i in range(n):
    photoz_train.append(model_variational(np.array([train_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).mean()[0][0])

photoz_train = np.asarray(photoz_train).astype('float32')

photoz_validation = []

for i in range(v):
    photoz_validation.append(model_variational(np.array([val_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).mean()[0][0])

photoz_validation = np.asarray(photoz_validation).astype('float32')

In [ ]:
specz = np.asarray(y_test)[:,0][:t]
# specz_train = np.asarray(y_train)[:,0][:n]
# specz_validation = np.asarray(y_validation)[:,0][:v]

photoz_err = []

for i in range(t):
    photoz_err.append(model_variational(np.array([test_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).stddev()[0][0])

photoz_err_train = []

for i in range(n):
    photoz_err_train.append(model_variational(np.array([train_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).stddev()[0][0])

photoz_err_validation = []

for i in range(v):
    photoz_err_validation.append(model_variational(np.array([val_gen[int(i/BATCH_SIZE)][0][int(i%BATCH_SIZE)]])).stddev()[0][0])

In [ ]:
from scipy.stats import gaussian_kde
xy = np.asarray(np.vstack([specz, photoz])).astype('float32')
z = gaussian_kde(xy)(xy)

In [ ]:
import seaborn as sns
z_max = 4
sns.set(rc={'figure.figsize': (10, 10), 'lines.markersize': 20})
plt.rcParams.update({'font.size': 40})
fig, ax = plt.subplots()
scatter_plot = ax.scatter(specz, photoz, c = z, s = 1, edgecolor = None)
plt.colorbar(scatter_plot, label = 'Density')
plt.title('Prediction')
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([0, z_max], [0, z_max],color = 'black')
plt.xlim([0, z_max])
plt.ylim([0, z_max])
plt.show()

sns.set(rc={'figure.figsize':(10, 10), 'lines.markersize':20})
plt.rcParams.update({'font.size': 100})
sns.set(font_scale=3)

data = {'BCNN':10.7, 'BNN':19.7, 'Mizuki':27.4,
        'DEmP':25}
models_list = list(data.keys())
outlier_rates = list(data.values())

plt.bar(models_list, outlier_rates, color=('blue', 'red', 'red', 'red'))
plt.plot([-.5, 3.5], [15, 15], color = 'black', linestyle = 'dashed')
plt.title("Outlier Rates")
plt.ylabel("% Outlier Rate")
plt.xlabel("Model Type")
plt.show()

In [ ]:
overlap_variational = 0
overlap_array_variational = []
for i in range(0,t):
    if np.abs(specz[i]-photoz[i])<=photoz_err[i]:
        overlap_variational += 1
        overlap_array_variational.append(1)
    else:
        overlap_array_variational.append(0)

In [ ]:
fig, ax = plt.subplots()
display_size = 1000
scatter_plot = ax.scatter(specz[:display_size], photoz[:display_size], c = z[:display_size], s = 4, edgecolor = None, zorder = 2)
error_plot = ax.errorbar(specz[:display_size], photoz[:display_size], yerr = photoz_err[:display_size], fmt = "o", markersize = 0, color = 'blue', elinewidth = 1, zorder = 1)
plt.title('Prediction with error, Bayesian')
plt.xlabel('spectroscopic redshift')
plt.ylabel('photo z')
plt.plot([0, z_max], [0, z_max], color = 'black')
plt.xlim([0, z_max])
plt.ylim([0, z_max])
print("Coverage: "+str(overlap_variational/t))

In [ ]:
combined_array_variational = np.transpose(np.vstack((specz, photoz, photoz_err, overlap_array_variational))) 

In [ ]:
sorted_combined_array_variational = sorted(combined_array_variational, key=lambda x: x[0])

In [ ]:
bins_variational = int(z_max*5)
splitted_sorted_combined_array_variational = np.array_split(sorted_combined_array_variational, bins_variational)
coverage_variational = []
for i in range(0, bins_variational):
    bins_count_variational = 0
    for j in range(0,len(splitted_sorted_combined_array_variational[i])):
        if splitted_sorted_combined_array_variational[i][j][3] == 1:
            bins_count_variational += 1
    coverage_variational.append(bins_count_variational/len(splitted_sorted_combined_array_variational[i])/0.6827)
x_array = np.arange(0, z_max, z_max/bins_variational)

In [ ]:
plt.plot(x_array, coverage_variational, c = 'red')
plt.title('Coverage, Bayesian')
plt.xlabel('spectroscopic redshift')
plt.ylabel('coverage')
plt.ylim([0, 1.5])
plt.plot([0, 4], [1, 1], color = 'black', linestyle = 'dashed')
print("Coverage: "+str(overlap_variational/t))

In [ ]:
mean_array = []
for i in range(0, bins_variational):
    total = 0
    for j in range(0, len(splitted_sorted_combined_array_variational[i])):
        bias = (splitted_sorted_combined_array_variational[i][j][1] - splitted_sorted_combined_array_variational[i][j][0]) / (1 + splitted_sorted_combined_array_variational[i][j][0])
        total += bias/(1 + splitted_sorted_combined_array_variational[i][j][0])
    mean_array.append(total/len(splitted_sorted_combined_array_variational[i]))
x_array = np.arange(0, z_max, z_max/bins_variational)

In [ ]:
error_array = []
for i in range(0,bins_variational):
    error_total = 0
    for j in range(0,len(splitted_sorted_combined_array_variational[i])):
        error_total += splitted_sorted_combined_array_variational[i][j][2]
    error_array.append(error_total/len(splitted_sorted_combined_array_variational[i]))

In [ ]:
sns.set(rc={'figure.figsize':(5, 5), 'lines.markersize':40})
plt.rcParams.update({'font.size': 40})
plt.errorbar(x_array, mean_array, yerr = error_array, fmt = "o", color = 'blue', markersize = 4, elinewidth = 2)
plt.title('Bias Plot for Probabilistic Model')
plt.xlabel('True Redshift')
plt.ylabel('Bias (Predicted - True)')
plt.plot([0, z_max], [0, 0], color = 'black')
plt.plot([0, z_max], [0.003, 0.003], color = 'black', linestyle = 'dashed')
plt.plot([0, z_max], [-0.003, -0.003], color = 'black', linestyle = 'dashed')
plt.ylim([-0.1, 0.1])
plt.show()

In [ ]:
combined_outlier_array = np.transpose(np.vstack((specz, photoz, photoz_err)))
sorted_combined_outlier_array = sorted(combined_outlier_array, key=lambda x: x[0])
bins_outlier = int(z_max*20)
splitted_sorted_combined_outlier_array = np.array_split(sorted_combined_outlier_array,bins_outlier)
outlier_array = []
outlier_count_total = 0
for i in range(0,bins_outlier):
    outlier_count = 0
    for j in range(0,len(splitted_sorted_combined_outlier_array[i])):
        if np.abs(splitted_sorted_combined_outlier_array[i][j][1] - splitted_sorted_combined_outlier_array[i][j][0]) >= 3 * splitted_sorted_combined_outlier_array[i][j][2]:
            outlier_count += 1
            outlier_count_total += 1
    outlier_array.append(outlier_count/len(specz)*bins_outlier)
x_array_outlier = np.arange(0,z_max,z_max/bins_outlier)
plt.plot(x_array_outlier, outlier_array, c = 'blue')
plt.title('Outlier Rate')
plt.xlabel('spec z')
plt.ylabel('outlier rate')
plt.xlim([0,z_max])
plt.ylim([0,1/2])
plt.show()

print("Outlier Rate: "+str(outlier_count_total/len(specz)))

In [ ]:
combined_outlier_array = np.transpose(np.vstack((specz,np.subtract(photoz, specz))))
sorted_combined_outlier_array = sorted(combined_outlier_array, key=lambda x: x[0])
bins_outlier = int(z_max*10)
splitted_sorted_combined_outlier_array = np.array_split(sorted_combined_outlier_array,bins_outlier)
outlier_array = []
outlier_count_total = 0
for i in range(0,bins_outlier):
    outlier_count = 0
    for j in range(0,len(splitted_sorted_combined_outlier_array[i])):
        if np.abs(splitted_sorted_combined_outlier_array[i][j][1]/(splitted_sorted_combined_outlier_array[i][j][0] + 1)) >= .15:
            outlier_count += 1
            outlier_count_total += 1
    outlier_array.append(outlier_count/len(specz)*bins_outlier)
x_array_outlier = np.arange(0,z_max,z_max/bins_outlier)
plt.plot(x_array_outlier, outlier_array, c = 'blue')
plt.title('Outlier Rate')
plt.xlabel('spec z')
plt.ylabel('outlier rate')
plt.xlim([0,z_max])
plt.ylim([0,1/2])
plt.show()

print("Outlier Rate: "+str(outlier_count_total/len(specz)))

In [ ]:
photoz_utils.save_with_oid_std(model_name, object_id, specz, photoz, photoz_err)
# photoz_utils.save_train_with_oid_std(model_name, object_id_train, specz_train, photoz_train, photoz_err_train)
# photoz_utils.save_validation_with_oid_std(model_name, object_id_validation, specz_validation, photoz_validation, photoz_err_validation)

In [ ]:
uncertains = []
for i in range(0, len(photoz_err)):
    if photoz_err[i].numpy() >= 1:
        uncertains.append(i)
        print("Object ID: " + str(object_id[i]) + ", Spectro_z: " + str(specz[i]) + ", Photo_z: " + str(photoz[i]) + ", Photo_z_err: " + str(photoz_err[i].numpy()))

In [ ]:
check = 2
current = uncertains[check]
print("Object ID: " + str(object_id[current]) + ", Spectro_z: " + str(specz[current]) + ", Photo_z: " + str(photoz[current]) + ", Photo_z_err: " + str(photoz_err[current].numpy()))
plt.imshow(x_test[current][:,:,0])
plt.grid(False)

In [ ]:
biases = []
for i in range(0, len(photoz)):
    if np.abs(photoz[i]-specz[i]) >= 1 and photoz_err[i].numpy() <= 0.5:
        biases.append(i)
        print("Object ID: " + str(object_id[i]) + ", Spectro_z: " + str(specz[i]) + ", Photo_z: " + str(photoz[i]) + ", Photo_z_err: " + str(photoz_err[i].numpy()))

In [ ]:
check_bias = 3
current_bias = biases[check_bias]
print("Object ID: " + str(object_id[current_bias]) + ", Spectro_z: " + str(specz[current_bias]) + ", Photo_z: " + str(photoz[current_bias]) + ", Photo_z_err: " + str(photoz_err[current_bias].numpy()))
plt.imshow(x_test[current_bias][:,:,0])
plt.grid(False)

In [ ]:
# END